<a href="https://colab.research.google.com/github/JuanMaFB/JuanMaFB.github.io/blob/master/Copia_de_Copia_de_Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Trabajo Final (Predictiva): Pronostico de la evolución de casos activos de SARS-CoV-2 en Colombia
El trabajo que se presenta a continuación tiene como objetivo la predicción de corto y mediano plazo de el total de casos confirmados, los nuevos casos, los casos activos, recuperados y muertes para las 5 principales ciudades de Colombia, utilizando técnicas estadísticas, de inteligencia artificial o modelos híbridos..

El dataset que se utilizará es Casos positivos de COVID-19 en Colombia y se encuentra en www.datos.gov.co..

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


## Descripción de las variables disponibles
Antes de realizar cualquier cálculo con un nuevo conjunto de datos, es importante realizar una exploración descriptiva de los mismos. Esto permitirá comprender la información que contiene cada variable y dar indicios sobre qué variables serían adecuadas usar o no en etapas posteriores de análisis.

Con base a la información facilitada en www.datos.gov.co., las variables en este conjunto de datos disponibles tenemos:


1.  ***id_de_caso:***
2.   **fecha_de_notificaci_n**:Fecha de notificación a SIVIGILA
3.  **c_digo_divipola:**
4.  **ciudad_de_ubicaci_n:**
5.  **departamento:**
6.  **atenci_n:**
7.  **edad:**
8.  **sexo:** 
9.  **tipo:**
10.  **estado**: Corresponde a muertes no relacionadas con COVID-19, aún si eran casos activos **Hay pacientes recuperados para COVID-19, que pueden permanecer en hospitalización por otras comorbilidades
11.  **pa_s_de_procedencia**:
12.  **fis**: Fecha de inicio de síntomas
13.  **fecha_de_muerte**:
14.  **fecha_diagnostico**: Fecha de confirmación por laboratorio
15.  **fecha_recuperado**:
16.  **fecha_reporte_web**: Fecha de publicación en sitio web
17.  **tipo_recuperaci_n**: 	
Se refiere a la variable de tipo de recuperación que tiene dos opciones: PCR y tiempo. PCR indica que la persona se encuentra recuperada por segunda muestra, en donde dio negativo para el virus; mientras que tiempo significa que son personas que cumplieron 30 días posteriores al inicio de síntomas o toma de muestras que no tienen síntomas, que no tengan más de 70 años ni que estén hospitalizados.
18.  **codigo_departamento**:
19.  **codigo_pais**:
20.  **pertenencia_etnica**: Esta variable se actualizará cada semana.
21.  **nombre_grupo_etnico**:

### *Lectura de la base de datos*
Procedemos a importar las librerías a utilizar y llamamos la base de datos:

In [ ]:
!pip install pandas
!pip install sodapy

In [ ]:
## librerías
import pandas as pd
from sodapy import Socrata
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
cliente = Socrata("www.datos.gov.co", None)
df = cliente.get("gt2j-8ykr", limit=1000000)

# Convertir los datos a un DataFrame
df_0 = pd.DataFrame.from_records(df)


In [ ]:
df_1= df_0.copy()
#df1 = pd.read_csv("https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv?accessType=DOWNLOAD")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,14,16,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## *Exploración de la base de datos*

A continuación analizamos las características de la base de datos buscando describir cómo se relaciona cada atributo con la variable objetivo

In [ ]:
## Tamaño de la base de datos
print("La base de datos contiene {} registros  y {} columnas o atributos.".format(df_0.shape[0], df_0.shape[1]))
## Primeros 5 registros
df_1.head()

La base de datos contiene 572270 registros  y 21 columnas o atributos.


,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,ITALIA,2020-02-27T00:00:00.000,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,PCR,11,380,Otro,NaN,NaN
1,2,2020-03-06T00:00:00.000,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,ESPAÑA,2020-03-04T00:00:00.000,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,PCR,76,724,Otro,NaN,NaN
2,3,2020-03-07T00:00:00.000,05001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,ESPAÑA,2020-02-29T00:00:00.000,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,PCR,5,724,Otro,NaN,NaN
3,4,2020-03-09T00:00:00.000,05001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,NaN,2020-03-06T00:00:00.000,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,NaN
4,5,2020-03-09T00:00:00.000,05001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,NaN,2020-03-08T00:00:00.000,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,NaN


In [ ]:
#Columnas de la base de datos
print('Nombre columnas:',df_1.columns)

Nombre columnas: Index(['id_de_caso', 'fecha_de_notificaci_n', 'c_digo_divipola',
       'ciudad_de_ubicaci_n', 'departamento', 'atenci_n', 'edad', 'sexo',
       'tipo', 'estado', 'pa_s_de_procedencia', 'fis', 'fecha_diagnostico',
       'fecha_recuperado', 'fecha_reporte_web', 'tipo_recuperaci_n',
       'codigo_departamento', 'codigo_pais', 'pertenencia_etnica',
       'nombre_grupo_etnico', 'fecha_de_muerte'],
      dtype='object')


Revisando la base de datos y debido al gran numero de valores nulos que tienen la variables (codigo_pais) y a la poca importancia que genera en la base de datos se toma la decisión de dejarla por fuera del analisis

In [ ]:
 df0=df_1[['id_de_caso', 'fecha_de_notificaci_n', 'c_digo_divipola',
       'ciudad_de_ubicaci_n', 'departamento', 'atenci_n', 'edad', 'sexo',
       'tipo', 'estado', 'pa_s_de_procedencia', 'fis', 'fecha_diagnostico', 'fecha_reporte_web', 'tipo_recuperaci_n',
       'codigo_departamento', 'pertenencia_etnica',
       'nombre_grupo_etnico',]]

#, 'fecha_de_muerte' , 'fecha_recuperado'   

Verificamos si todas las columnas tienen igual registros y tambien se puede ver que las columnas de la base de datos seleccionada contiene atributos del siguiente tipo objects:

In [ ]:
# tipo de datos
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572270 entries, 0 to 572269
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id_de_caso             572270 non-null  object        
 1   fecha_de_notificaci_n  572270 non-null  datetime64[ns]
 2   c_digo_divipola        572270 non-null  object        
 3   ciudad_de_ubicaci_n    572270 non-null  object        
 4   departamento           572270 non-null  object        
 5   atenci_n               572270 non-null  object        
 6   edad                   572270 non-null  object        
 7   sexo                   572270 non-null  object        
 8   tipo                   572270 non-null  object        
 9   estado                 572270 non-null  object        
 10  pa_s_de_procedencia    965 non-null     object        
 11  fis                    572270 non-null  object        
 12  fecha_diagnostico      568427 non-null  date

Podemos observar que todas las columnas son de tipo object por lo tanto se deben de modificar las columnas  *fecha_de_notificaci_n, fecha_diagnostico, fecha_recuperado, fecha_reporte_web y fecha_de_muerte* a un tipo de datos ***datetime***, las columnas *id_de_caso y  edad* se deben de modificar a tipo de datos ***int64*** y el resto de columnas se modifican al tipo de datos ***category***


In [ ]:
df0['fecha_de_notificaci_n']= pd.to_datetime(df0['fecha_de_notificaci_n'])
df0['fecha_diagnostico']= pd.to_datetime(df0['fecha_diagnostico'])
#df0['fecha_recuperado']= pd.to_datetime(df0['fecha_recuperado'])
df0['fecha_reporte_web']= pd.to_datetime(df0['fecha_reporte_web'])
#df0['fecha_de_muerte']= pd.to_datetime(df0['fecha_de_muerte'])
df0['id_de_caso']= df0['id_de_caso'].values.astype(float)
df0['edad']= df0['edad'].values.astype(float)

df0 = df0.astype({'c_digo_divipola':'category', 'ciudad_de_ubicaci_n':'category',
       'departamento':'category', 'atenci_n':'category', 'edad':'int64',  'id_de_caso':'int64','sexo':'category',
       'tipo':'category', 'estado':'category', 'pa_s_de_procedencia':'category', 'fis':'category',
       'codigo_departamento':'category', 'tipo_recuperaci_n':'category'})

In [ ]:
#Validamos el cambio de tipo de datos
df0.dtypes

id_de_caso                        int64
fecha_de_notificaci_n    datetime64[ns]
c_digo_divipola                category
ciudad_de_ubicaci_n            category
departamento                   category
atenci_n                       category
edad                              int64
sexo                           category
tipo                           category
estado                         category
pa_s_de_procedencia            category
fis                            category
fecha_diagnostico        datetime64[ns]
fecha_reporte_web        datetime64[ns]
tipo_recuperaci_n              category
codigo_departamento            category
pertenencia_etnica               object
nombre_grupo_etnico              object
dtype: object

Identificamos el % de valores perdidos

In [ ]:
# Funcion para calcular los valores perdidos por columna
def Valores_perdidos(df):
          
        # Total valores perdidos 
        mis_val = df.isnull().sum()
        
        # % de valores perdidos 
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Crear una tabla con los resultados
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Renombrar las columnas
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Ordenar la tabla con los valores más altos de forma ascendente
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Imprimir la informacion
        print ("El Dataframe tiene " + str(df.shape[1]) + " columnas.\n"      
            "hay " + str(mis_val_table_ren_columns.shape[0]) +
              " a las que le faltan valores.")
        
        # Retornar el dataframe
        return mis_val_table_ren_columns

In [ ]:
Valores_perdidos(df0)

El Dataframe tiene 18 columnas.
hay 5 a las que le faltan valores.


,Missing Values,% of Total Values
pa_s_de_procedencia,571305,99.8
nombre_grupo_etnico,564131,98.6
pertenencia_etnica,291989,51.0
tipo_recuperaci_n,165149,28.9
fecha_diagnostico,3843,0.7


Encontramos 5 variables con valores nulos por lo tanto analizaremos una a una para ver como podemos imputar los valores nulos


***pais de procedencia*** los valores nulos corresponden a los Nacionales reemplazaremos por Colombia.
Se observa que el pais de ARABIA, ESTADOS UNIDOS, MEXICO,, PANAMÁ Y PERU  esta escrita de formas diferentes esto podria afectar el analisis, por este motivo se deben de igualar

In [ ]:
df0.groupby(('pa_s_de_procedencia')).id_de_caso.count()

pa_s_de_procedencia
ALEMANIA                                            12
ANTILLAS NEERLANDESAS                                8
ARABIA SAUDITA                                       1
ARABIA SAUDÍ                                         1
ARGENTINA                                            5
ARUBA                                                3
AUSTRALIA                                            1
BRASIL                                              64
BÉLGICA                                              1
CANADÁ                                               3
CHILE                                               20
COSTA RICA                                           2
CROACIA                                              1
CUBA                                                 4
CURAZAO                                              2
ECUADOR                                             68
EGIPTO                                              12
EMIRATOS ARABES UNIDOS                       

In [ ]:
df0['pa_s_de_procedencia'] = df0.pa_s_de_procedencia.replace("ARABIA SAUD", "ARABIA SAUDITA")
df0['pa_s_de_procedencia'] = df0.pa_s_de_procedencia.replace("ESTADOS UNIDOS", "ESTADOS UNIDOS DE AMÉRICA")
df0['pa_s_de_procedencia'] = df0.pa_s_de_procedencia.replace("MEXICO ", "MÉXICO")
df0['pa_s_de_procedencia'] = df0.pa_s_de_procedencia.replace("PANAMA", "PANAMÁ")
df0['pa_s_de_procedencia'] = df0.pa_s_de_procedencia.replace("PERU", "PERÚ")
df0['pa_s_de_procedencia'] = df0['pa_s_de_procedencia'].cat.add_categories('COLOMBIA')
df0['pa_s_de_procedencia'].fillna('COLOMBIA', inplace =True) 

Para la variable **fecha_diagnostico** los valores nulos suman 3844, analizando esta fecha con la fecha de registro es diferente solo en 127 registros por lo reemplazaremos la fecha de gestión con la fehca de reporte web ya que esta ultima no tiene valores nulos

In [ ]:
# restamos las 2 fechas con el fin de validar cuantos valores superan el 0
dfcolum=df0[['fecha_diagnostico','fecha_reporte_web','fecha_de_notificaci_n']]
dfcolum['dfcolumvalor']=df0['fecha_diagnostico']-df0['fecha_reporte_web']

In [ ]:
dfcolum.head(10)

,fecha_diagnostico,fecha_reporte_web,fecha_de_notificaci_n,dfcolumvalor
0,2020-03-06,2020-03-06,2020-03-02,0 days
1,2020-03-09,2020-03-09,2020-03-06,0 days
2,2020-03-09,2020-03-09,2020-03-07,0 days
3,2020-03-11,2020-03-11,2020-03-09,0 days
4,2020-03-11,2020-03-11,2020-03-09,0 days
5,2020-03-11,2020-03-11,2020-03-10,0 days
6,2020-03-11,2020-03-11,2020-03-08,0 days
7,2020-03-11,2020-03-11,2020-03-09,0 days
8,2020-03-11,2020-03-11,2020-03-08,0 days
9,2020-03-12,2020-03-12,2020-03-12,0 days


In [ ]:
# solo 127 registros tienen fecha diferente 
dfcolum.groupby(('dfcolumvalor')).dfcolumvalor.max()

dfcolumvalor
-122 days   -122 days
-108 days   -108 days
-101 days   -101 days
-100 days   -100 days
-97 days     -97 days
               ...   
35 days       35 days
46 days       46 days
50 days       50 days
53 days       53 days
66 days       66 days
Name: dfcolumvalor, Length: 127, dtype: timedelta64[ns]

In [ ]:
df0["fecha_diagnostico"] = df0["fecha_reporte_web"]

Para la variable **tipo_recuperaci_n**  los valores nulos suman 166.658, los nulos los reemplazaremos por No identificado

In [ ]:
df0['tipo_recuperaci_n'] = df0['tipo_recuperaci_n'] .cat.add_categories('No identificado')
df0['tipo_recuperaci_n'].fillna('No identificado', inplace =True) 

Para la variable **pertenencia_etnica** los valores nulos suman 281.847, los nulos los reemplazaremos por Otro

In [ ]:
#df0['pertenencia_etnica'] = df0['pertenencia_etnica'].cat.add_categories('Otro')
df0['pertenencia_etnica'].fillna('Otro', inplace =True) 

Para la variable **nombre_grupo_etnico** los valores nulos suman 553989, los nulos los reemplazaremos por No identificado

In [ ]:
#df0['nombre_grupo_etnico'] = df0['nombre_grupo_etnico'] .cat.add_categories('No identificado')
df0['nombre_grupo_etnico'].fillna('No identificado', inplace =True) 


Verificamos columna a columna que los datos esten correctos

In [ ]:
#df1 = df0.groupby(('tipo', 'atenci_n')).id_de_caso.count()
df0.groupby(('tipo_recuperaci_n')).id_de_caso.count()

tipo_recuperaci_n
PCR                 87655
Tiempo             319466
No identificado    165149
Name: id_de_caso, dtype: int64

In [ ]:
#df1 = df0.groupby(('tipo', 'atenci_n')).id_de_caso.count()
df0.groupby(('pertenencia_etnica')).id_de_caso.count()

df0['pertenencia_etnica'].fillna('Otro', inplace =True) 

In [ ]:
df0.groupby(('tipo')).id_de_caso.count()

tipo
En estudio     542037
Importado         958
RELACIONADO        14
Relacionado     29223
relacionado        38
Name: id_de_caso, dtype: int64

Se observa que la palabra Relacionado esta escrita de 3 formas diferentes esto podria afectar el analisis, por este motivo se deben de igualar

In [ ]:
df0['tipo'] = df0.tipo.replace("RELACIONADO", "Relacionado")
df0['tipo'] = df0.tipo.replace("relacionado", "Relacionado")

In [ ]:
df0.groupby(('departamento')).id_de_caso.count()

departamento
Amazonas                                                     2693
Antioquia                                                   74796
Arauca                                                        788
Archipiélago de San Andrés Providencia y Santa Catalina       172
Atlántico                                                   27698
Barranquilla D.E.                                           35732
Bogotá D.C.                                                199324
Bolívar                                                      4537
Boyacá                                                       3395
Buenaventura D.E.                                            2395
Caldas                                                       2547
Caquetá                                                      5206
Cartagena D.T. y C.                                         19839
Casanare                                                      930
Cauca                                                        45

Se observa que hay departamentos que tienen como nombre su capital, se modifican los valores por el nombre del Departamento real

In [ ]:
df0.departamento.replace("Barranquilla D.E.", "Atlántico")
df0.departamento.replace("Cartagena D.T. y C.", "Bolívar")
df0.departamento.replace("Santa Marta D.T. y C.", "Magdalena")
df0.departamento.replace("Buenaventura D.E.", "Valle del Cauca")
df0.departamento.replace("Bogotá D C","Bogotá D.C") 

0             Bogotá D.C.
1         Valle del Cauca
2               Antioquia
3               Antioquia
4               Antioquia
               ...       
572265          Antioquia
572266          Antioquia
572267          Antioquia
572268          Antioquia
572269          Antioquia
Name: departamento, Length: 572270, dtype: category
Categories (37, object): [Amazonas, Antioquia, Arauca,
                          Archipiélago de San Andrés Providencia y Santa..., ..., Tolima, Valle del Cauca, Vaupés, Vichada]

In [ ]:
df0.groupby(('atenci_n')).id_de_caso.count()

atenci_n
Casa            123765
Fallecido        18184
Hospital         20176
Hospital UCI      1853
N/A               1171
Recuperado      407121
Name: id_de_caso, dtype: int64

Se observa que hay 1.171 registros esto podria afectar el analisis, por este motivo se deben modificar

In [ ]:
df0['atenci_n'] = df0.atenci_n.replace("CASA", "Casa")
df0.groupby(('atenci_n')).id_de_caso.count()

atenci_n
Casa            123765
Fallecido        18184
Hospital         20176
Hospital UCI      1853
N/A               1171
Recuperado      407121
Name: id_de_caso, dtype: int64

In [ ]:
df0.groupby(('sexo')).id_de_caso.count()

sexo
F    277448
M    294813
f         7
m         2
Name: id_de_caso, dtype: int64

Se observa que la palabra F y M esta escrita de 2 formas diferentes esto podria afectar el analisis, por este motivo se deben de igualar

In [ ]:
df0['sexo'] = df0.sexo.replace("f", "F")
df0['sexo'] = df0.sexo.replace("m", "M")
df0.groupby(('sexo')).id_de_caso.count()

sexo
F    277455
M    294815
Name: id_de_caso, dtype: int64

In [ ]:
df0.groupby(('estado')).id_de_caso.count()

estado
Asintomático     66536
Fallecido        18184
Grave             2014
Leve            462449
Moderado         21853
N/A               1234
Name: id_de_caso, dtype: int64

Se observa que hay 1.171 registros esto podria afectar el analisis, por este motivo se deben modificar

In [ ]:
# Buscamos fecha maxima
min(df0['fecha_de_notificaci_n'])

Timestamp('2020-03-02 00:00:00')

In [ ]:
# Buscamos fecha minima
max(df0['fecha_de_notificaci_n'])

Timestamp('2020-08-26 00:00:00')

In [ ]:
df0['estado'] = df0.estado.replace("LEVE", "Leve")
df0.groupby(('estado')).id_de_caso.count()

estado
Asintomático     66536
Fallecido        18184
Grave             2014
Leve            462449
Moderado         21853
N/A               1234
Name: id_de_caso, dtype: int64

Se observa que la palabra ANDOQUE Y ZENÚ esta escrita de 2 formas diferentes esto podria afectar el analisis, por este motivo se deben de igualar




In [ ]:
df0.groupby(df0['fecha_de_notificaci_n'].dt.month).id_de_caso.count()

fecha_de_notificaci_n
3      1912
4      6992
5     30609
6     95706
7    263598
8    173453
Name: id_de_caso, dtype: int64

In [ ]:
df0.describe()

,id_de_caso,edad
count,572270.000000,572270.000000
mean,286175.193947,39.622274
std,165200.789683,18.125528
min,1.000000,0.000000
25%,143108.250000,27.000000
50%,286175.500000,37.000000
75%,429242.750000,52.000000
max,572310.000000,110.000000


Las columnas de la base de datos seleccionada contiene atributos del siguiente tipo:

In [ ]:
df0.isnull().sum().sum()

0

Observamos que hay una persona en la base de datos que tiene 116 años, por lo tanto se debe validar. Nota pedir orientación de que hacer con este registro.

In [ ]:
df = df0[df0['edad']>109]
df

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,pertenencia_etnica,nombre_grupo_etnico
466413,466454,2020-08-01,47001,Santa Marta,Santa Marta D.T. y C.,Recuperado,110,F,En estudio,Leve,COLOMBIA,2020-07-29T00:00:00.000,2020-08-16,2020-08-16,Tiempo,47,Otro,No identificado


In [ ]:
print(df0.isnull().sum(axis = 0))
#df0_tr.groupby(('estado')).tipo_recuperaci_n.count()

id_de_caso               0
fecha_de_notificaci_n    0
c_digo_divipola          0
ciudad_de_ubicaci_n      0
departamento             0
atenci_n                 0
edad                     0
sexo                     0
tipo                     0
estado                   0
pa_s_de_procedencia      0
fis                      0
fecha_diagnostico        0
fecha_reporte_web        0
tipo_recuperaci_n        0
codigo_departamento      0
pertenencia_etnica       0
nombre_grupo_etnico      0
dtype: int64


In [ ]:
ruta = "covid_ff.csv"
df0.to_csv(ruta)